# AltumAge

## Index
1. [Instantiate model class](#Instantiate-model-class)
2. [Define clock metadata](#Define-clock-metadata)
3. [Download clock dependencies](#Download-clock-dependencies)
5. [Load features](#Load-features)
6. [Load weights into base model](#Load-weights-into-base-model)
7. [Load reference values](#Load-reference-values)
8. [Load preprocess and postprocess objects](#Load-preprocess-and-postprocess-objects)
10. [Check all clock parameters](#Check-all-clock-parameters)
10. [Basic test](#Basic-test)
11. [Save torch model](#Save-torch-model)
12. [Clear directory](#Clear-directory)

Let's first import some packages:

In [1]:
import os
import inspect
import shutil
import json
import torch
import pandas as pd
import pyaging as pya
import tensorflow as tf
from tensorflow.keras.models import load_model

## Instantiate model class

In [2]:
def print_entire_class(cls):
    source = inspect.getsource(cls)
    print(source)

print_entire_class(pya.models.AltumAge)

class AltumAge(pyagingModel):
    def __init__(self):
        super().__init__()

    def preprocess(self, x):
        """
        Scales an array based on the median and standard deviation.
        """
        median = torch.tensor(
            self.preprocess_dependencies[0], device=x.device, dtype=x.dtype
        )
        std = torch.tensor(
            self.preprocess_dependencies[1], device=x.device, dtype=x.dtype
        )
        x = (x - median) / std
        return x

    def postprocess(self, x):
        return x



In [3]:
model = pya.models.AltumAge()

## Define clock metadata

In [4]:
model.metadata["clock_name"] = 'altumage'
model.metadata["data_type"] = 'methylation'
model.metadata["species"] = 'Homo sapiens'
model.metadata["year"] = 2022
model.metadata["approved_by_author"] = '✅'
model.metadata["citation"] = "de Lima Camillo, Lucas Paulo, Louis R. Lapierre, and Ritambhara Singh. \"A pan-tissue DNA-methylation epigenetic clock based on deep learning.\" npj Aging 8.1 (2022): 4."
model.metadata["doi"] = 'https://doi.org/10.1038/s41514-022-00085-y'
model.metadata["research_only"] = None
model.metadata["notes"] = None

## Download clock dependencies

#### Download GitHub repository

In [5]:
github_url = "https://github.com/rsinghlab/AltumAge.git"
github_folder_name = github_url.split('/')[-1].split('.')[0]
os.system(f"git clone {github_url}")

0

## Load features

In [6]:
model.features = pd.read_pickle('AltumAge/example_dependencies/multi_platform_cpgs.pkl').tolist()

## Load weights into base model

In [7]:
AltumAge = load_model('AltumAge/example_dependencies/AltumAge.h5')  # Load your trained TensorFlow model
weights = {}
for layer in AltumAge.layers:
    weights[layer.name] = layer.get_weights()

base_model = pya.models.AltumAgeNeuralNetwork()

# Function to copy weights from TensorFlow to PyTorch
def copy_weights(torch_layer, tf_weights, bn=False):
    with torch.no_grad():
        if bn:
            torch_layer.weight.data = torch.tensor(tf_weights[0]).float()
            torch_layer.bias.data = torch.tensor(tf_weights[1]).float()
            torch_layer.running_mean.data = torch.tensor(tf_weights[2]).float()
            torch_layer.running_var.data = torch.tensor(tf_weights[3]).float()
        else:
            torch_layer.weight.data = torch.tensor(tf_weights[0]).T.float()
            torch_layer.bias.data = torch.tensor(tf_weights[1]).float()

# Now copy the weights
copy_weights(base_model.bn1, weights['batch_normalization_84'], bn=True)
copy_weights(base_model.linear1, weights['dense_84'])
copy_weights(base_model.bn2, weights['batch_normalization_85'], bn=True)
copy_weights(base_model.linear2, weights['dense_85'])
copy_weights(base_model.bn3, weights['batch_normalization_86'], bn=True)
copy_weights(base_model.linear3, weights['dense_86'])
copy_weights(base_model.bn4, weights['batch_normalization_87'], bn=True)
copy_weights(base_model.linear4, weights['dense_87'])
copy_weights(base_model.bn5, weights['batch_normalization_88'], bn=True)
copy_weights(base_model.linear5, weights['dense_88'])
copy_weights(base_model.bn6, weights['batch_normalization_89'], bn=True)
copy_weights(base_model.linear6, weights['dense_89'])

model.base_model = base_model

## Load reference values

In [8]:
scaler = pd.read_pickle('AltumAge/example_dependencies/scaler.pkl')

model.reference_values = scaler.center_

## Load preprocess and postprocess objects

In [9]:
model.preprocess_name = 'scale'
model.preprocess_dependencies = [scaler.center_, scaler.scale_]

In [10]:
model.postprocess_name = None
model.postprocess_dependencies = None

## Check all clock parameters

In [11]:
pya.utils.print_model_details(model)


%==================================== Model Details ====================================%
Model Attributes:

training: True
metadata: {'approved_by_author': '✅',
 'citation': 'de Lima Camillo, Lucas Paulo, Louis R. Lapierre, and Ritambhara '
             'Singh. "A pan-tissue DNA-methylation epigenetic clock based on '
             'deep learning." npj Aging 8.1 (2022): 4.',
 'clock_name': 'altumage',
 'data_type': 'methylation',
 'doi': 'https://doi.org/10.1038/s41514-022-00085-y',
 'notes': None,
 'research_only': None,
 'species': 'Homo sapiens',
 'version': None,
 'year': 2022}
reference_values: array([0.7598634 , 0.78637881, 0.06324422, ..., 0.03556449, 0.04053195,
       0.05189659])
preprocess_name: 'scale'
preprocess_dependencies: [array([0.7598634 , 0.78637881, 0.06324422, ..., 0.03556449, 0.04053195,
       0.05189659]),
 array([0.18540869, 0.42506826, 0.03971112, ..., 0.0264798 , 0.01924175,
       0.03057686])]
postprocess_name: None
postprocess_dependencies: None
features

## Basic test

In [12]:
torch.manual_seed(42)
input = torch.randn(10, len(model.features), dtype=float)
model.eval()
model.to(float)
pred = model(input)
pred

tensor([[ 1.6940e+01],
        [-7.8034e-02],
        [ 1.2900e+02],
        [ 3.2433e+01],
        [ 8.1607e+01],
        [ 3.2819e+01],
        [ 7.1175e+01],
        [ 4.5454e+01],
        [ 3.1396e+01],
        [ 1.9619e+02]], dtype=torch.float64, grad_fn=<AddmmBackward0>)

## Save torch model

In [13]:
torch.save(model, f"../weights/{model.metadata['clock_name']}.pt")

## Clear directory
<a id="10"></a>

In [14]:
# Function to remove a folder and all its contents
def remove_folder(path):
    try:
        shutil.rmtree(path)
        print(f"Deleted folder: {path}")
    except Exception as e:
        print(f"Error deleting folder {path}: {e}")

# Get a list of all files and folders in the current directory
all_items = os.listdir('.')

# Loop through the items
for item in all_items:
    # Check if it's a file and does not end with .ipynb
    if os.path.isfile(item) and not item.endswith('.ipynb'):
        os.remove(item)
        print(f"Deleted file: {item}")
    # Check if it's a folder
    elif os.path.isdir(item):
        remove_folder(item)

Deleted folder: AltumAge
Deleted folder: .ipynb_checkpoints
